In [3]:
!pip install pyomo
!apt-get install -y -qq glpk-utils

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.6/49.6 kB 5.0 MB/s eta 0:00:00
Selecting previously unselected package libsuitesparseconfig5:amd64.
(Reading database ... 120828 files and directories currently installed.)
Preparing to unpack .../libsuitesparseconfig5_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libsuitesparseconfig5:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libamd2:amd64.
Preparing to unpack .../libamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libcolamd2:amd64.
Preparing to unpack .../libcolamd2_1%3a5.10.1+dfsg-4build1_amd64.deb ...
Unpacking libcolamd2:amd64 (1:5.10.1+dfsg-4build1) ...
Selecting previously unselected package libglpk40:amd64.
Preparing to unpack .../libglpk40_5.0-1_amd64.deb ...
Unpacking libglpk40:amd64 (5.0-1) ...
Selecting previously unselected pac

In [4]:
import pyomo.environ as pyo
from pyomo.opt import SolverFactory

In [5]:
# Example 1
# Defining the model
model1 = pyo.ConcreteModel()

# Decision variables
model1.x1 = pyo.Var(within = pyo.NonNegativeReals)
x1 = model1.x1
model1.x2 = pyo.Var(within = pyo.NonNegativeReals)
x2 = model1.x2

# Objective function
model1.objective = pyo.Objective(expr = 4*x1 + 3*x2, sense = pyo.maximize)

# Conctraints
model1.constraint1 = pyo.Constraint(expr = x1 + x2 <= 40)
model1.constraint2 = pyo.Constraint(expr = 2*x1 + x2 <= 60)

optm = SolverFactory('glpk')
results = optm.solve(model1)
print(results)
print('objective function = ', model1.objective())
print('x1 variable = ', x1())
print('x2 variable = ', x2())



Problem: 
- Name: unknown
  Lower bound: 140.0
  Upper bound: 140.0
  Number of objectives: 1
  Number of constraints: 2
  Number of variables: 2
  Number of nonzeros: 4
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.005213260650634766
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

objective function =  140.0
x1 variable =  20.0
x2 variable =  20.0


In [14]:
# Example 2
# Defining the model
model2 = pyo.ConcreteModel()

# Defining index set
model2.N = pyo.Set(initialize = ['Desk','Table','Chair'])

# Parameters
model2.L = pyo.Param(model2.N, initialize = {'Desk':8,'Table':6,'Chair':1})
L = model2.L
model2.F = pyo.Param(model2.N, initialize = {'Desk':4,'Table':2,'Chair':1.5})
F = model2.F
model2.C = pyo.Param(model2.N, initialize = {'Desk':2,'Table':1.5,'Chair':0.5})
C = model2.C
model2.P = pyo.Param(model2.N, initialize = {'Desk':60,'Table':30,'Chair':20})
P = model2.P

# Decision variables
model2.x = pyo.Var(model2.N, within = pyo.NonNegativeReals)

# Objective function
def objective_rule(model2):
  return sum(P[i]*model2.x[i] for i in model2.N)

model2.objective = pyo.Objective(rule = objective_rule, sense = pyo.maximize)

# Constraints
def constraint1(model2,i):
  return sum(L[i]*model2.x[i] for i in model2.N) <= 48

model2.const1 = pyo.Constraint(rule = constraint1 )

def constraint2(model2,i):
  return sum(F[i]*model2.x[i] for i in model2.N) <= 20

model2.const2 = pyo.Constraint(rule = constraint2 )

def constraint3(model2,i):
  return sum(C[i]*model2.x[i] for i in model2.N) <= 8

model2.const3 = pyo.Constraint(rule = constraint3 )

def constraint4(model2,i):
  return model2.x['Table'] <= 5
model2.const4 = pyo.Constraint(rule = constraint4)

# Solving the model
solver = SolverFactory('glpk')
result = solver.solve(model2)

# Showing the values of solution
print(result)
print('Objetive function =', model2.objective())
for i in model2.N:
  print('Number of',i, 'produced =', model2.x[i]())




Problem: 
- Name: unknown
  Lower bound: 280.0
  Upper bound: 280.0
  Number of objectives: 1
  Number of constraints: 4
  Number of variables: 3
  Number of nonzeros: 10
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.006577014923095703
Solution: 
- number of solutions: 0
  number of solutions displayed: 0

Objetive function = 280.0
Number of Desk produced = 2.0
Number of Table produced = 0.0
Number of Chair produced = 8.0
